In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/GoogleA14/AI4Code (1).zip') as z:
  z.extractall()

In [3]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('/content')

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/GoogleA14/train_df.csv')
val_df = pd.read_csv('/content/drive/MyDrive/GoogleA14/val_df.csv')

In [5]:
import re
from nltk.stem import WordNetLemmatizer
from pathlib import Path
import nltk
nltk.download('wordnet')

stemmer = WordNetLemmatizer()
nltk.download('omw-1.4')
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()
        #return document

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)
        return preprocessed_text

train_df.source = train_df.source.apply(preprocess_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [8]:
train_df

,Unnamed: 0,index,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,0,0,00001756c60be8,1862f0a6,code,this python environment come with many helpful analytics library installed defined kaggle python docker image http g...,0,945aea18,NaN,0.000000
1,1,1,00001756c60be8,2a9e43d6,code,import numpy import panda import random from sklearn model_selection import train_test_split cross_val_score from sk...,2,945aea18,NaN,0.034483
2,2,2,00001756c60be8,038b763d,code,import warning warning filterwarnings ignore,4,945aea18,NaN,0.068966
3,3,3,00001756c60be8,2eefe0ef,code,matplotlib rcparams update font size,6,945aea18,NaN,0.103448
4,4,4,00001756c60be8,0beab1cd,code,evaluate_preds train_true_values train_pred_values test_true_values test_pred_values print train round train_true_va...,8,945aea18,NaN,0.137931
...,...,...,...,...,...,...,...,...,...,...
5740827,5740827,6370645,fffe1d764579d5,0d770d6b,markdown,removing outlier,43,3c40bfa6,NaN,0.597222
5740828,5740828,6370646,fffe1d764579d5,d45ddc62,markdown,dimensionality curse,33,3c40bfa6,NaN,0.458333
5740829,5740829,6370647,fffe1d764579d5,1a63248d,markdown,bangalore house price prediction,0,3c40bfa6,NaN,0.000000
5740830,5740830,6370648,fffe1d764579d5,a8ffc8b4,markdown,have achieved accuracy predicting price home banglore using decision tree regressor,69,3c40bfa6,NaN,0.958333


In [ ]:
# import os
# if not os.path.exists('text.txt'):
#   with open('/content/drive/MyDrive/GoogleA14/checkpoint/text.txt','w') as f:
#     for id, item in tqdm(train_df.groupby('id')):
#       df_markdown =  item[item['cell_type']=='markdown']
#       for source, rank in df_markdown[['source', 'rank']].values:
#         cell_source = df_markdown[df_markdown['rank']==(rank+1)]
#         if len(cell_source):
#           setence = source + ' [SEP] ' + cell_source.source.values[0]
#           f.write(setence+'\n')
      

In [11]:
try:
  import transformers
except:
  !pip install transformers

from transformers import AutoModel, AutoTokenizer, DataCollatorForLanguageModeling, AutoModelWithLMHead
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset

#os.mkdir('/content/drive/MyDrive/GoogleA14/checkpoint')
checkpoint_path = '/content/drive/MyDrive/GoogleA14/checkpoint'
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/GoogleA14/codebert-base')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/GoogleA14/codebert-base')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

from transformers import Trainer, TrainingArguments

dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = '/content/drive/MyDrive/GoogleA14/checkpoint/text.txt',
    block_size = 128  # maximum sequence length
)

print('No. of lines: ', len(dataset)) # No of lines in your datset

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/GoogleA14/checkpoint/result',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    save_steps=20000,
    save_strategy = 'steps',
    seed = 0
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
trainer.train()
trainer.save_model('/content/drive/MyDrive/GoogleA14/checkpoint/model')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file /content/drive/MyDrive/GoogleA14/codebert-base/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/MyDrive/GoogleA14/codebert-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output

No. of lines:  479113


Step,Training Loss
500,6.164300
1000,4.521100
1500,4.086700
2000,3.895000
2500,3.775000
3000,3.667000
3500,3.608700
4000,3.521000
4500,3.486400
5000,3.471800


Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-20000
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-20000/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-40000
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-40000/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-40000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-60000
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-60000/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-60000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/c

Step,Training Loss
500,6.164300
1000,4.521100
1500,4.086700
2000,3.895000
2500,3.775000
3000,3.667000
3500,3.608700
4000,3.521000
4500,3.486400
5000,3.471800


Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-120000
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-120000/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-120000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-140000
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-140000/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/result/checkpoint-140000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/GoogleA14/checkpoint/model
Configuration saved in /content/drive/MyDrive/GoogleA14/checkpoint/model/config.json
Model weights saved in /content/drive/MyDrive/GoogleA14/checkpoint/model/pytorch_model.bin
